In [1]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [4]:
augment_df = preprocess.pd.read_csv(r"D:\MScDataScience\7.Data_Science_Project\SourceCode\augment_data\EDA\eda_augmented_data_size_5.csv", index_col=0)

In [6]:
preprocess.pd.concat([preprocess.train_df, augment_df])

,Query,Intent
0,can you walk me through setting up direct depo...,direct_deposit
1,i want to switch to direct deposit,direct_deposit
2,set up direct deposit for me,direct_deposit
3,how do i go about setting up direct deposit,direct_deposit
4,i need to get my paycheck direct deposited to ...,direct_deposit
...,...,...
37495,can increase i to 700 dollars my credit limit,credit_limit_change
37496,can i increase my credit limit to 700 dollars,credit_limit_change
37497,can i increase my course credit fix to 700 dol...,credit_limit_change
37498,can i increase my credit limit to dollar 700 d...,credit_limit_change
